In [ ]:
%%capture
%run ./index.ipynb

### Static Penalty

When using a penalty-based approach, then values with possibly different scales are aggregated. Thus, bringing the objectives and constraints to the same scale is recommended to create a meaningful new optimization problem. The scale of the constraints has already been discussed in [here](index.ipynb). The scaling of the objective is shown below:

In [ ]:
from pymoo.constraints.tcv import TotalConstraintViolation

tcv = TotalConstraintViolation(ieq_eps=0.0,
                               ieq_pow=1.0,
                               ieq_scale=ieq_scale,
                               eq_eps=0.0,
                               eq_pow=None,
                               eq_scale=eq_scale,
                               aggr_func=np.mean)

Placeholder

In [ ]:
# extract inequality and equality constraints
F = sample.get("F")

# we also need to scale the objective
f_scale = np.max(F) - np.min(F)

# this can be tricky: the lowest objective value we think can be achieved
f_ideal = 1.0

Next, we define the actual optimization problem. *pymoo* already provides a problem class to define the penalty:

In [ ]:
from pymoo.constraints.tcv import TotalConstraintViolation
from pymoo.constraints.penalty import ConstraintsAsPenalty

# the penalty factor - how much more does cv count in comparison to f
penalty = 10.0

penalty_problem = ConstraintsAsPenalty(problem, 
                                       tcv, 
                                       f_ideal=f_ideal, 
                                       f_scale=f_scale, 
                                       penalty=penalty)


Then, we solve the unconstrained optmization problem using an optimizer:

In [ ]:
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.optimize import minimize

algorithm = DE(
    pop_size=100,
    sampling=LHS(),
    variant="DE/rand/1/bin",
    CR=0.3,
    dither="vector",
    jitter=False
)

res = minimize(penalty_problem,
               algorithm,
               seed=1,
               verbose=False)

The solution we have obtained is one of the unconstrained problems. There is no guarantee that this has indeed found a feasible solution to the original problem.

In [ ]:
from pymoo.core.individual import Individual
opt = Individual(X=res.X)
Evaluator().eval(problem, opt)

print("Best solution found: \nX = %s\nF = %s\nCV = %s" % (opt.X, opt.F, opt.CV))

The obtained solution seems to be slightly infeasible. This happens because the algorithm does not know feasible or infeasible solutions and just optimizes the combined objective function. Some more iterations on the original problem can help to make the obtained solution feasible.